In [104]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.interaction import KEY
from selenium.webdriver.common import keys
import time

In [ ]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [105]:
class VideoBot:
    
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.driver.get("https://www.youtube.com/watch?v=xBYHFv1zDKc&list=RDxBYHFv1zDKc&start_radio=1")
        self.driver.maximize_window()
        bot = self.driver
        time.sleep(3)
        actions = ActionChains(self.driver)
        actions.send_keys('f')
        actions.perform()
    def change(self):
        driver = self.driver
        actions = ActionChains(driver)
        actions.key_down(keys.Keys.SHIFT)
        actions.send_keys("n")
        actions.key_up(keys.Keys.SHIFT)
        actions.perform()
        

In [ ]:
model = model_from_json(open("fer.json", "r").read())

In [ ]:
model.load_weights('fer.h5')

In [ ]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
cap=cv2.VideoCapture(0)

In [ ]:
Bot = VideoBot()

In [ ]:
while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=4)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]
        if predicted_emotion == 'angry':
            Bot.change()
    
        
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows